![New Relic](./assets/new-relic-logo.png)

# New Relic Playground

## Imports

Python Imports

In [ ]:
import json
import os

from textwrap import dedent

Third-party libraries

In [ ]:
import dotenv
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np

from requests import Session

## Useful definitions

In [ ]:
NULL_CURSOR = json.dumps(None)

In [ ]:
def build_query(query_string, query_params = None):
    if query_params is None:
        query_params = {}
        
    return dedent(query_string.strip()) % query_params

In [ ]:
def get_response_data(response, key_path = None, action = "actor"):
    data = response.json().get("data").get(action)

    if key_path is not None:
        for key in key_path.split(":"):
            if key.isdecimal() and isinstance(data, list):
                data = data[int(key)]
            else:
                data = data.get(key)

    return data

In [ ]:
class NewRelicGqlClient(Session):
    """Client for New Relic GraphQL API."""

    url: str = "https://api.newrelic.com/graphql"

    def __init__(self, new_relic_user_key = None):
        super().__init__()

        self.headers.update(
            {
                "Content-Type": "application/json",
                "API-Key": new_relic_user_key,
            }
        )

    def execute(self, query, variables = None, **kwargs):
        data = json.dumps(
            {
                "query": query,
                "variables": variables,
            },
        )
        return self.post(self.url, data=data, **kwargs)

## Client setup

Load credentials

In [ ]:
env_file = "../.env"

dotenv.load_dotenv(env_file)

new_relic_user_key = os.environ.get("NEW_RELIC_USER_KEY", None)

if new_relic_user_key is None:
    raise ValueError("Environment variable NEW_RELIC_USER_KEY is not set.")

Client instantiation

In [ ]:
newrelic = NewRelicGqlClient(new_relic_user_key)

Define test query

In [ ]:
query = build_query(
    """
        {
            actor {
                user {
                    email
                    name
                    id
                }
            }
        }
    """
)

Executing query

In [ ]:
response = newrelic.execute(query)

Looking for response data

In [ ]:
print(
    json.dumps(
        get_response_data(response, action="actor"),
        indent=4
    )
)

## Playground area

From now on you can use the client defined in the previous section to make your queries to the New Relic GraphQL API

In [ ]:
# Put your code here :D
